In [1]:
from nltk.translate.bleu_score import SmoothingFunction
from bert_score import score
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu
import evaluate
import json
import nltk
from nltk.tokenize import word_tokenize
nltk.download('wordnet')
from tqdm import tqdm
smoothie = SmoothingFunction().method4
rouge = evaluate.load('rouge')
messages_final = json.load(open('data/messages_final.json'))
messages_boh_original_final = json.load(open('data/messages_boh_original_final.json'))
messages_no_sw_original_final = json.load(open('data/messages_no_sw_original_final.json'))
test_data = json.load(open('data/test_data.json'))

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/csgrad/sunilruf/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import os
import torch
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"
os.environ['TRANSFORMERS_CACHE'] = '/home/csgrad/sunilruf/'
path = "/data/sunilruf/llama/fp16_no/rank_32_1/checkpoint-50/"
torch.cuda.empty_cache()
tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True, device_map="auto")

In [8]:
len(test_data)

66

In [9]:
len(messages_final[0])

22

In [3]:
from datasets import load_dataset

dataset = load_dataset("thu-coai/esconv")

In [5]:

bleu_2_final, bleu_2_boh_original_final, bleu_2_no_sw_original_final = [], [], []
bleu_4_final, bleu_4_boh_original_final, bleu_4_no_sw_original_final = [], [], []
bert_score_final, bert_score_boh_original_final, bert_score_no_sw_original_final = [], [], []
rouge_final, rouge_boh_original_final, rouge_no_sw_original_final = [], [], []
meteor_final, meteor_boh_original_final, meteor_no_sw_original_final = [], [], []

for j in tqdm(range(3, 53)):
    data = json.loads(dataset['test'][j]['text'])
    data = data['dialog']
    j = j-3
    try:
        for i in range(0, len(data),1):
            #print(i)
            if data[i+1]['speaker'] == "sys":
                reference = data[i+1]['text']
            elif data[i+2]['speaker'] == "sys":
                reference = data[i+1]['text']
            elif data[i+3]['speaker'] == "sys":
                reference = data[i+1]['text']
            #print(messages_final[j][i]['content'])
            
            bleu_2_final.append(sentence_bleu(messages_final[j][i]['content'], reference, weights=(0.5,0.5), smoothing_function=smoothie))
            bleu_2_boh_original_final.append(sentence_bleu(messages_boh_original_final[j][i]['content'], reference, weights=(0.5,0.5), smoothing_function=smoothie))
            bleu_2_no_sw_original_final.append(sentence_bleu(messages_no_sw_original_final[j][i]['content'], reference, weights=(0.5,0.5), smoothing_function=smoothie))
            bleu_4_final.append(sentence_bleu(messages_final[j][i]['content'], reference, weights=(0.25,0.25,0.25,0.25), smoothing_function=smoothie))
            bleu_4_boh_original_final.append(sentence_bleu(messages_boh_original_final[j][i]['content'], reference, weights=(0.25,0.25,0.25,0.25), smoothing_function=smoothie))
            bleu_4_no_sw_original_final.append(sentence_bleu(messages_no_sw_original_final[j][i]['content'], reference, weights=(0.25,0.25,0.25,0.25), smoothing_function=smoothie))
            bert_score_final.append(score([messages_final[j][i]['content']], [reference], lang='en', model_type='bert-base-uncased'))
            bert_score_boh_original_final.append(score([messages_boh_original_final[j][i]['content']], [reference], lang='en', model_type='bert-base-uncased'))
            bert_score_no_sw_original_final.append(score([messages_no_sw_original_final[j][i]['content']], [reference], lang='en', model_type='bert-base-uncased'))
            rouge_final.append(rouge.compute(predictions=[messages_final[j][i]['content']], references=[reference]))
            rouge_boh_original_final.append(rouge.compute(predictions=[messages_boh_original_final[j][i]['content']], references=[reference]))
            rouge_no_sw_original_final.append(rouge.compute(predictions=[messages_no_sw_original_final[j][i]['content']], references=[reference]))
            meteor_final.append((nltk.translate.meteor_score.meteor_score([word_tokenize(messages_final[j][i]['content'])], word_tokenize(reference))))
            meteor_boh_original_final.append((nltk.translate.meteor_score.meteor_score([word_tokenize(messages_boh_original_final[j][i]['content'])], word_tokenize(reference))))
            meteor_no_sw_original_final.append((nltk.translate.meteor_score.meteor_score([word_tokenize(messages_no_sw_original_final[j][i]['content'])], word_tokenize(reference))))
            
    except Exception as e:
        print(e)

  2%|▏         | 1/50 [00:31<25:37, 31.37s/it]

list index out of range
list index out of range


  6%|▌         | 3/50 [01:34<24:41, 31.52s/it]

list index out of range


  8%|▊         | 4/50 [02:18<27:23, 35.74s/it]

...

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range


 88%|████████▊ | 44/50 [12:33<00:39,  6.65s/it]

list index out of range
list index out of range
list index out of range


100%|██████████| 50/50 [13:53<00:00, 16.66s/it]

list index out of range
list index out of range
list index out of range
list index out of range


In [6]:
rougel_final = [i['rougeL'] for i in rouge_final]
rougel_boh_original_final = [i['rougeL'] for i in rouge_boh_original_final]
rougel_no_sw_original_final = [i['rougeL'] for i in rouge_no_sw_original_final]

In [7]:
bert_p_boh = [i[0].item() for i in bert_score_boh_original_final]
bert_r_boh = [i[1].item() for i in bert_score_boh_original_final]
bert_f1_boh = [i[2].item() for i in bert_score_boh_original_final]

bert_p_no_sw = [i[0].item() for i in bert_score_no_sw_original_final]
bert_r_no_sw = [i[1].item() for i in bert_score_no_sw_original_final]
bert_f1_no_sw = [i[2].item() for i in bert_score_no_sw_original_final]

bert_p = [i[0].item() for i in bert_score_final]
bert_r = [i[1].item() for i in bert_score_final]
bert_f1 = [i[2].item() for i in bert_score_final]

In [9]:
import statistics

print("Bleu 2 Final: ", statistics.mean(bleu_2_final))
print("Bleu 2 Boh Original Final: ", statistics.mean(bleu_2_boh_original_final))
print("Bleu 2 No Sw Original Final: ", statistics.mean(bleu_2_no_sw_original_final))
print("Bleu 4 Final: ", statistics.mean(bleu_4_final))
print("Bleu 4 Boh Original Final: ", statistics.mean(bleu_4_boh_original_final))
print("Bleu 4 No Sw Original Final: ", statistics.mean(bleu_4_no_sw_original_final))
print("Rouge L Final: ", statistics.mean(rougel_final))
print("Rouge L Boh Original Final: ", statistics.mean(rougel_boh_original_final))
print("Rouge L No Sw Original Final: ", statistics.mean(rougel_no_sw_original_final))
print("Bert Precision Final: ", statistics.mean(bert_p))
print("Bert Recall Final: ", statistics.mean(bert_r))
print("Bert F1 Final: ", statistics.mean(bert_f1))
print("Bert Precision Boh Original Final: ", statistics.mean(bert_p_boh))
print("Bert Recall Boh Original Final: ", statistics.mean(bert_r_boh))
print("Bert F1 Boh Original Final: ", statistics.mean(bert_f1_boh))
print("Bert Precision No Sw Original Final: ", statistics.mean(bert_p_no_sw))
print("Bert Recall No Sw Original Final: ", statistics.mean(bert_r_no_sw))
print("Bert F1 No Sw Original Final: ", statistics.mean(bert_f1_no_sw))
print("Meteor Final: ", statistics.mean(meteor_final))
print("Meteor Boh Original Final: ", statistics.mean(meteor_boh_original_final))
print("Meteor No Sw Original Final: ", statistics.mean(meteor_no_sw_original_final))


Bleu 2 Final:  0.04990642951830513
Bleu 2 Boh Original Final:  0.04851962966414015
Bleu 2 No Sw Original Final:  0.047713156153653
Bleu 4 Final:  0.012940582340584521
Bleu 4 Boh Original Final:  0.012747041182022047
Bleu 4 No Sw Original Final:  0.012608999242617762
Rouge L Final:  0.13146653834425245
Rouge L Boh Original Final:  0.12997184823008937
Rouge L No Sw Original Final:  0.10265179402352556
Bert Precision Final:  0.4097596603980486
Bert Recall Final:  0.43743031390424614
Bert F1 Final:  0.41896728313398146
Bert Precision Boh Original Final:  0.41039121300652287
Bert Recall Boh Original Final:  0.4116433668306385
Bert F1 Boh Original Final:  0.40698227002613546
Bert Precision No Sw Original Final:  0.39910804360792196
Bert Recall No Sw Original Final:  0.39881590462249256
Bert F1 No Sw Original Final:  0.3953047702158766
Meteor Final:  0.12588550722585168
Meteor Boh Original Final:  0.12392321106898453
Meteor No Sw Original Final:  0.11489665543036834


In [10]:
import torch

perplexity_final = torch.load('data/perplexity_final.pt')
perplexity_boh = torch.load('data/perplexity_boh_final.pt')
perplexity_sw = torch.load('data/perplexity_no_sw_final.pt')


In [11]:
perplexity_final1 = []
perplexity_final1.extend(per for sublist in perplexity_final for per in sublist )
perplexity_boh1 = []
perplexity_boh1.extend(per for sublist in perplexity_boh for per in sublist )
perplexity_sw1 = []
perplexity_sw1.extend(per for sublist in perplexity_sw for per in sublist )

print("Perplexity Final: ", torch.mean(torch.tensor(perplexity_final1)))
print("Perplexity Boh Original Final: ", torch.mean(torch.tensor(perplexity_boh1)))
print("Perplexity No Sw Original Final: ", torch.mean(torch.tensor(perplexity_sw1)))

Perplexity Final:  tensor(10.5679)
Perplexity Boh Original Final:  tensor(37.1390)
Perplexity No Sw Original Final:  tensor(35.2154)


In [26]:
import json
import torch
from tqdm import tqdm
import statistics
messages_boh = json.load(open('data/messages_boh_final.json'))
messages_sw = json.load(open('data/messages_no_sw_final.json'))
messages_final = json.load(open('data/messages_final.json'))

In [37]:
boh_length, sw_length, final_length = [], [], []
for j in tqdm(range(0,47)):
    for i in range(1,len(messages_final[j]),2):
        final_length.append(len(tokenizer.tokenize(messages_final[j][i]['content'])))
        sw_length.append(len(tokenizer.tokenize(messages_sw[j][i]['content'])))
        boh_length.append(len(tokenizer.tokenize(messages_boh[j][i]['content'])))
        
print("actual length ", statistics.mean(final_length))
print("boh length ", statistics.mean(boh_length))
print("sw length ", statistics.mean(sw_length))

100%|██████████| 47/47 [00:00<00:00, 362.24it/s]

actual length  56.229268292682924
boh length  23.507317073170732
sw length  18.390243902439025
